In [1]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time

In [ ]:
# portfolio of stocks
stocks = ['GOOGL', 'AMZN', 'AAPL', 'META', 'MSFT', 'NVDA', 'BABA', 'WMT', 'TSLA', 'DOCU', 'PANW', 'UNH', 'MSTR', 'ZM', 'MA', 'V', 'EBAY']

#### Download historical data from yf API

In [3]:
import yfinance as yf

start_date = "2020-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(stocks, start=start_date, end=end_date, group_by='ticker')

# download data to csv
data.to_csv("stocks.csv")

[*********************100%***********************]  13 of 13 completed


#### Download analysis from csv downloaded data

In [4]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("stocks.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Forward fill missing values for each column
df = df.fillna(method='ffill')

# Convert all values to float
df = df.astype(float)

# rename tickers to more readable names, ^DJI -> DJ, ^GSPC -> SP, ^IXIC -> NQ
df.columns = pd.MultiIndex.from_tuples([
    (
        col[0].replace('^DJI', 'DJ').replace('^GSPC', 'SP').replace('^IXIC', 'NQ').replace('^GDAXI', 'DAX').replace('^N225', 'NKY').replace('RTY=F', 'RUT'),
        col[1]
    ) for col in df.columns
])


# Show the result
df.head()

BABA                                                  \
                  Open        High         Low       Close      Volume   
Date                                                                     
2020-01-02  205.703946  208.913905  205.646952  208.714478  15873500.0   
2020-01-03  205.466521  207.226302  205.143613  206.083817   8604500.0   
2020-01-06  204.079962  206.235774  203.320203  205.741928  11885500.0   
2020-01-07  206.691621  207.926227  205.789413  206.682129   9388000.0   
2020-01-08  205.703941  209.550194  205.438028  207.033508  11959100.0   

              MSTR                                    ...       TSLA  \
              Open    High     Low   Close    Volume  ...       Open   
Date                                                  ...              
2020-01-02  14.284  14.489  14.272  14.431  276000.0  ...  28.299999   
2020-01-03  14.302  14.489  14.280  14.357  585000.0  ...  29.366667   
2020-01-06  14.325  14.546  14.325  14.425  327000.0  ...  29.364668   
2020-01-07  14.400  14.537  14.324  14.387  349000.0  ...  30.760000   
2020-01-08  14.395  14.547  14.207  14.371  395000.0  ...  31.580000   

                                                               AAPL  \
                 High        Low      Close       Volume       Open   
Date                                                                  
2020-01-02  28.713333  28.114000  28.684000  142981500.0  71.545882   
2020-01-03  30.266666  29.128000  29.534000  266677500.0  71.765674   
2020-01-06  30.104000  29.333332  30.102667  151995000.0  70.954203   
2020-01-07  31.441999  30.224001  31.270666  268231500.0  72.415345   
2020-01-08  33.232666  31.215334  32.809334  467164500.0  71.768086   

                                                          
                 High        Low      Close       Volume  
Date                                                      
2020-01-02  72.598884  71.292296  72.538506  135480400.0  
2020-01-03  72.594063  71.608692  71.833298  146322800.0  
2020-01-06  72.444336  70.703027  72.405693  118387200.0  
2020-01-07  72.671348  71.845377  72.065155  108872000.0  
2020-01-08  73.526303  71.768086  73.224411  132079200.0  

[5 rows x 65 columns]

In [5]:
# # Here you can plot the charts for every ticker

# # For every ticker, plot Close
# import matplotlib.pyplot as plt

# for ticker in df.columns.levels[0]:
#     plt.figure(figsize=(12, 6))
#     plt.plot(df[ticker]['Close'], label=ticker)
#     plt.title(f"{ticker} Close Price")
#     plt.xlabel("Date")
#     plt.ylabel("Close Price")
#     plt.legend()
#     plt.show()

#### Plot of Volume Profile for All Stocks

In [6]:
import plotly.io as pio
pio.renderers.default = 'vscode'
import plotly.express as px
import plotly.graph_objects as go

for stock in stocks:
    # Use the stock data from our df
    df_stock = df[stock][['Close', 'Volume']]

    # Convert the stock to datetime if needed
    if not isinstance(df_stock.index, pd.DatetimeIndex):
        df_stock.index = pd.to_datetime(df_stock.index)

    # plot
    hist = go.Histogram(
        y=df_stock['Close'],
        x=df_stock['Volume'],
        nbinsy=150,
        orientation='h',
        name='Volume Profile',
        marker_color='#B0C4DE',
        opacity=0.5,
        xaxis='x2',
    )

    line = go.Scatter(
        x=df_stock.index,
        y=df_stock['Close'],
        mode='lines',
        name='Close',
        marker_color='blue',
        xaxis='x',
        yaxis='y',
    )

    fig = go.Figure([hist, line])

    fig.update_layout(
        xaxis=dict(
            title='Date',
            anchor='y',
            domain=[0, 1],
            showgrid=False,
            side='bottom'
        ),
        xaxis2=dict(
            title='Sum of Volume',
            overlaying='x',
            side='top',
            showgrid=False
        ),
        yaxis=dict(
            title='Close Price',
            showgrid=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        height=600,
        title=f'Volume Profile & {stock} Close Prices'
    )

    fig.show()